In [ ]:
!pip install comet_ml

In [ ]:
import comet_ml #at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key:
experiment = Experiment(
    api_key="cjZUHKCBKcrudJIeYuUe1zaBT",
    project_name="species-audio-detection",
    workspace="kaggle",
    log_code=True,
)

In [ ]:
import os
import pathlib as pt

import cv2
import librosa
import librosa.display
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import pandas as pd 
import signal
import soundfile

from itertools import repeat

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from fastai.vision.all import *
from fastai.data.core import DataLoaders

from tqdm import tqdm

import torch.cuda
if torch.cuda.is_available():
    print('PyTorch found cuda')
else:
    print('PyTorch could not find cuda')

## Prepare data

In [ ]:
ROOT = pt.Path('/kaggle/input/rfcx-species-audio-detection')
TRAIN_TP_CSV = ROOT/"train_tp.csv"
TRAIN_FP_CSV = ROOT/"train_fp.csv"
TRAIN_DIR    = ROOT/"train"
TEST_DIR     = ROOT/"test"

print(list(TRAIN_DIR.glob("*"))[:5])

In [ ]:
def create_path(row):
    return TRAIN_DIR/"{}.flac".format(row)

In [ ]:
train_audio_tp_df = pd.read_csv(TRAIN_TP_CSV)
train_audio_tp_df["tp"] = True
train_audio_tp_df['audio_path'] = train_audio_tp_df['recording_id'].apply(create_path)
train_audio_tp_df.head(3)

In [ ]:
train_audio_fp_df = pd.read_csv(TRAIN_FP_CSV)
train_audio_fp_df["tp"] = False
train_audio_fp_df['audio_path'] = train_audio_fp_df['recording_id'].apply(create_path)
train_audio_fp_df.head(3)

In [ ]:
print("TP: # {} FP: # {}".format(len(train_audio_tp_df), len(train_audio_fp_df)))

In [ ]:
all_train_audio_df = pd.concat([train_audio_tp_df, train_audio_fp_df]).reset_index()
all_train_audio_df.head(3)

In [ ]:
species_ids = sorted(all_train_audio_df['species_id'].unique())
print(species_ids)

In [ ]:
_df = all_train_audio_df['species_id'].value_counts().sort_index()
ax = _df.plot(kind='bar')
ax.set_xlabel("Species ID")
ax.set_ylabel("Frequency")
ax.set_title("Nr of samples / species")
plt.close('all')

In [ ]:
test_audio_paths = list(TEST_DIR.glob("*.flac"))
test_audio_df = pd.DataFrame()
test_audio_df['recording_id'] = [p.stem for p in test_audio_paths]
test_audio_df['audio_path']   = [p for p in test_audio_paths]
test_audio_df.head(3)

In [ ]:
print("Total training: # {}".format(len(all_train_audio_df)))
print("Test: # {}".format(len(test_audio_df)))

### Split train data into train and validation

In [ ]:
# train_df, valid_df = train_test_split(all_train_df, test_size=0.2, random_state=42)
# train_df.reset_index(drop=True, inplace=True)
# valid_df.reset_index(drop=True, inplace=True)

# print("Train: # {} Validation: # {}".format(len(train_df), len(valid_df)))

### Audio to spectogram

In [ ]:
# idx = 256
# y, sr = librosa.load(all_train_audio_df.iloc[idx]['audio_path'])
# fig, ax = plt.subplots(1, 1)
# ax.plot(y);
# ax.set_title('Signal - recording id {}'.format(all_train_audio_df.iloc[idx]['recording_id']));
# ax.set_xlabel('Time (samples)');
# ax.set_ylabel('Amplitude');

In [ ]:
# spec = np.abs(librosa.stft(y, hop_length=512))
# spec = librosa.amplitude_to_db(spec, ref=np.max)
# fig, ax = plt.subplots(1, 1)
# img = librosa.display.specshow(spec, sr=sr, x_axis='time', y_axis='log', ax=ax);
# ax.set_title('Spectrogram');
# fig.colorbar(img, ax=ax, format='%+2.0f dB');

In [ ]:
# mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
# fig, ax = plt.subplots(1, 1)
# img = librosa.display.specshow(librosa.power_to_db(mel_spec, ref=np.max), y_axis='mel', x_axis='time')
# ax.set_title('Melspectogram');
# fig.colorbar(img, ax=ax, format='%+2.0f dB');

# plt.close('all')

In [ ]:
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf).convert('RGB')
 
    return img

In [ ]:
# %matplotlib
# %matplotlib

# fig, ax = plt.subplots(1, 1, figsize=(0.72, 0.72))
# img = librosa.display.specshow(librosa.power_to_db(mel_spec, ref=np.max), y_axis='mel', x_axis='time')
# ax.set_axis_off()
# ax.axis('tight')
# # Set whitespace to 0
# fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
# # img = fig2img(fig)
# # img.save("pic_melspectogram_example_3.png")
# fig.savefig('melspectogram_example.png', dpi=400)#, bbox_inches='tight', pad_inches=0)
# experiment.log_image('melspectogram_example.png')

### Save spectograms

In [ ]:
def save_melspectograms(data_df, output_dir, dim_inches=(0.72, 0.72)):
    for _, row in data_df.iterrows():
        if 'species_id' in data_df.columns:
            image_filename = "s{}_{}_train.png".format(row['species_id'], row['recording_id'])
        else:
            image_filename = "{}_test.png".format(row['recording_id'])
        melspec_path = output_dir/image_filename
        
        if melspec_path.exists():
            continue
        y, sr = librosa.load(row['audio_path'])

        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)

        fig, ax = plt.subplots(1, 1, figsize=dim_inches)
        _ = librosa.display.specshow(mel_spec, y_axis='mel', x_axis='time')
        ax.set_axis_off()
        ax.axis('tight')
        # Set whitespace to 0
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
        fig.savefig(melspec_path, dpi=400)#, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        plt.close('all')
        del y
        del mel_spec

    print("Done saving from idx {}.".format(data_df.index[0]))

In [ ]:
TRAIN_SPEC_DIR = pt.Path('/kaggle/working/train_specs/')
TEST_SPEC_DIR = pt.Path('/kaggle/working/test_specs/')
os.makedirs(TRAIN_SPEC_DIR, exist_ok=True)
os.makedirs(TEST_SPEC_DIR, exist_ok=True)

print("TRAIN_SPEC_DIR: {} TEST_SPEC_DIR: {}".format(TRAIN_SPEC_DIR.exists(), TEST_SPEC_DIR.exists()))

In [ ]:
def signal_handler(signal, frame):
    global interrupted
    interrupted = True
    
def parallelised_saving(data_df, output_dir, batch_nr=2):
    master_list = []
    num_proc = mp.cpu_count()

    n_iter = int(np.ceil(len(data_df) / batch_nr))
    for i in range(n_iter):
        batch_df = data_df.iloc[i * batch_nr: (i+1) * batch_nr]
        master_list.append(batch_df)

    print("to be saved dfs #", len(master_list))
    print("CPU cores #", num_proc)

    signal.signal(signal.SIGINT, signal_handler)

    processes = [None] * num_proc
    processed_idx = []
    local_current_index = 0

    interrupted = False
    finished = False
    exit_main_loop = False

    while(not interrupted and not exit_main_loop):

        for i in range(num_proc):

            if processes[i] is None and not finished:
                batch_idx = local_current_index
#                 print("Process number {} is free. Assigning new task.".format(i))
                if batch_idx >= len(master_list):
                    finished = True
                else:
                    processed_idx.append(batch_idx)
#                     print("\tStarting processing of batch nr {}".format(batch_idx))
                    processes[i] = mp.Process(target=save_melspectograms, 
                                              args=([master_list[batch_idx], output_dir]))
                    processes[i].start()
                    local_current_index = local_current_index + 1

            if(finished and all(v is None for v in processes)):
                print(">>> Finished processing!")
                exit_main_loop = True
                break

            if processes[i] is None:
                continue

            if processes[i].exitcode is not None:

                processes[i].join()
                processes[i] = None

        time.sleep(0.1)

In [ ]:
parallelised_saving(all_train_audio_df, output_dir=TRAIN_SPEC_DIR, batch_nr=64) # include FP data as well
# parallelised_saving(train_audio_tp_df, output_dir=TRAIN_SPEC_DIR, batch_nr=64)

In [ ]:
parallelised_saving(test_audio_df, output_dir=TEST_SPEC_DIR, batch_nr=64)

In [ ]:
train_melspectograms = list(TRAIN_SPEC_DIR.glob("*.png"))
print("Train total: #", len(train_melspectograms))

test_melspectograms = list(TEST_SPEC_DIR.glob("*.png"))
print("Test: #", len(test_melspectograms))

In [ ]:
# _i = Image.open(train_melspectograms[0])
# print(_i.shape)
# plt.imshow(_i)

## Train
### Prepare data loaders

In [ ]:
train_specs_df = pd.DataFrame()
train_specs_df['spec_path']    = train_melspectograms
train_specs_df['recording_id'] = [p.name.split("_")[1] for p in train_melspectograms]
train_specs_df['species_id']   = [p.name.split("_")[0] for p in train_melspectograms]
train_specs_df.head(2)

In [ ]:
test_specs_df = pd.DataFrame()
test_specs_df['spec_path']    = test_melspectograms
test_specs_df['recording_id'] = [p.name.split("_")[0] for p in test_melspectograms]
test_specs_df.head(2)

In [ ]:
data_loaders = ImageDataLoaders.from_df(train_specs_df, path="/", seed=42, fn_col='spec_path', label_col='species_id', item_tfms=[Resize(224)])

### Setup resnet34 architecture

In [ ]:
# creating directories and copying the models to those directories
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/resnet34/resnet34.pth /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
!cp ../input/resnet50/resnet50.pth /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
# !cp ../input/resnet152/resnet152.pth /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth

In [ ]:
learn = cnn_learner(data_loaders, resnet50, metrics=error_rate)
# learn.model = learn.model.cuda()
learn.lr_find()

In [ ]:
learn.fine_tune(10, base_lr=3e-3)
# learn.fit_one_cycle(n_epoch=2, lr_max=0.01)
# learn.export("resnet34_model.pkl")

In [ ]:
# experiment.log_model(name="resnet34_model_v0", file_or_folder="/resnet34_model.pkl")

## Look at some predictions

In [ ]:
learn.show_results()

In [ ]:
interp = Interpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15,10))

## Create submission file

In [ ]:
test_dl = data_loaders.test_dl(test_specs_df)
res_preds = learn.get_preds(dl=test_dl, with_decoded=True) # returns (predictions, _, predicted label)

In [ ]:
preds_values = res_preds[0]
preds_labels = res_preds[2]

In [ ]:
submission_data = {'recording_id': []}
for _id in species_ids:
    submission_data.update({'s{}'.format(_id): []})

for idx, preds in enumerate(preds_values):

    submission_data['recording_id'].append(test_specs_df.iloc[idx]['recording_id'])
    for i, p in enumerate(preds):
        submission_data["s{}".format(i)].append(p.item())

submission_df = pd.DataFrame(data=submission_data)
submission_df.to_csv("submission_data.csv", index=False)
experiment.log_table("submission_data.csv")

In [ ]:
submission_df.head()

In [ ]:
experiment.end()